In [2]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\oihater\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [14]:
import simpy
import random

# 시간 단위 : μs
# DataFrame = 1000byte, 각 station 전송속도는 항상 80Mbps 가정 -> 한개의 데이터프레임 당 100μs 소모(Transmission Delay)

cnt_load = []  # count load
cnt_succeed = []  # count transmission success

# data length = random.randint(10, 200) -> 데이터크기 100byte ~ 2000byte랜덤으로 생성, 전송시간 10~200μs

# 캐리어 센싱
def CarrierSensing(que):
    # idle
    if not que: #채널 큐가 비어있는지 비어있으면 idle 아니면 busy를 반환
        return 'idle'

    # busy
    else:
        return 'busy'

    
class Station:
    
    staID = 1  
    CS = ''

    def __init__(self, env):

        self.staID = Station.staID
        self.CS = Station.CS
        Station.staID += 1
        self.env = env
        env.process(self.run())
        
        
    def run(self):
        arrivalRate=0.01  # 평균 100μs 당 1개의 Data
        DataSend= random.expovariate(arrivalRate)  # 평균 100을 갖는 랜덤 포아송, 첫 데이터 전송시점
        trans_time=0.0
        
        # station beginning
        while True:
            datalength = random.randint(100, 1000) # 데이터 길이 -> 100~1000byte
            trans_time = datalength*8 / 80  # 전송시간, 전송속도 80Mbps로 고정
            yield env.timeout(DataSend)  # 데이터 전송시 채널 시점
            
            #캐리어 센싱
            self.CS = CarrierSensing(Channel.que) 
            print('Station %d : ' %self.staID, 'Carrier Sensing at : %f' %self.env.now, "Result : ", self.CS)
            
            cnt_load.append(self.staID)  # 데이터 도착
            
            if self.CS == 'busy':
                while True:
                    backoff = random.expovariate(1/trans_time)  # 랜덤시간 대기
                    yield env.timeout(backoff)
                    print('Station %d : ' %self.staID, 'back off to : %f' %self.env.now)

                    self.CS = CarrierSensing(Channel.que)  # 다시 캐리어 센싱
                    print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)

                    if self.CS == 'busy':  # 채널 전송 중인 경우 다시 backoff로
                        continue

                    elif self.CS == 'idle':  # 전송 채널 비어 있을 경우
                        while True:
                            backoff = random.uniform(0, 1)  # IFS 대기 후
                            yield env.timeout(backoff)
                            print('Station %d : ' %self.staID, 'wait for IFS to : %f' %self.env.now)
                            
                            self.CS = CarrierSensing(Channel.que)  # 다시 캐리어 센싱(동시 전송 방지)
                            print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)
                            if self.CS == 'idle':
                                Channel.que.append(self.staID)
                                print('Station %d : ' %self.staID, 'Transmission start at : %f' %self.env.now)
                                yield env.timeout(trans_time)

                                print('Station %d : ' %self.staID, 'Transmission complete(ACK) at : %f' %self.env.now)
                                cnt_succeed.append(self.staID)
                                Channel.que.clear()
                                break
                        break    
                            

            
            elif self.CS == 'idle':
                while True:
                    backoff = random.uniform(0, 1)  # IFS 대기 후
                    yield env.timeout(backoff)
                    print('Station %d : ' %self.staID, 'wait for IFS to : %f' %self.env.now)

                    self.CS = CarrierSensing(Channel.que)  # 다시 캐리어 센싱
                    print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)
                    if self.CS == 'idle':
                        Channel.que.append(self.staID)  
                        print('Station %d : ' %self.staID, 'Transmission start at : %f' %self.env.now)
                        yield env.timeout(trans_time)

                        print('Station %d : ' %self.staID, 'Transmission complete(ACK) at : %f' %self.env.now)
                        cnt_succeed.append(self.staID)
                        Channel.que.clear()  # 전송 완료 후 전송 채널 비우기
                        break

                    elif self.CS == 'busy':
                        backoff = random.expovariate(1/trans_time)  # 랜덤시간 대기
                        yield env.timeout(backoff)
                        print('Station %d : ' %self.staID, 'back off to : %f' %self.env.now)

                        self.CS = CarrierSensing(Channel.que)  # 다시 캐리어 센싱
                        print('Station %d : ' %self.staID, 'Carrier Sensing again at : %f' %self.env.now, "Result : ", self.CS)
            
            DataSend = random.expovariate(arrivalRate)  # 다음 데이터 전송시점   
                      
# channel
class Channel:
    que = []

    def __init__(self, env):
        Channel.succeedMsgEvt = env.event()
        Channel.failMsgEvt = env.event()
        Channel.idleMsgEvt = env.event()
        Channel.colCount = 0
        self.env = env
        env.process(self.run())

    def run(self):
        while True:
            Channel.nowtime += SIFsTime
            yield self.env.timeout(SIFsTime)   


env = simpy.Environment()
numofStation = 5 #스테이션 수
staSet = [Station(env) for i in range(numofStation)]
env.run(until=1000)

load = 0.0
for i in range(len(cnt_load)):
    load += 1
                                           
throughput = 0.0
for i in range(len(cnt_succeed)):
    throughput += 1                                           
                                          
print('throughput : ', throughput/load)

Station 2 :  Carrier Sensing at : 5.220340 Result :  idle
Station 2 :  wait for IFS to : 5.316206
Station 2 :  Carrier Sensing again at : 5.316206 Result :  idle
Station 2 :  Transmission start at : 5.316206
Station 1 :  Carrier Sensing at : 54.815215 Result :  busy
Station 2 :  Transmission complete(ACK) at : 56.016206
Station 3 :  Carrier Sensing at : 63.440407 Result :  idle
Station 3 :  wait for IFS to : 63.739199
Station 3 :  Carrier Sensing again at : 63.739199 Result :  idle
Station 3 :  Transmission start at : 63.739199
Station 1 :  back off to : 69.225111
Station 1 :  Carrier Sensing again at : 69.225111 Result :  busy
Station 1 :  back off to : 76.464336
Station 1 :  Carrier Sensing again at : 76.464336 Result :  busy
Station 4 :  Carrier Sensing at : 86.907828 Result :  busy
Station 1 :  back off to : 92.369631
Station 1 :  Carrier Sensing again at : 92.369631 Result :  busy
Station 1 :  back off to : 101.128579
Station 1 :  Carrier Sensing again at : 101.128579 Result :  bu